# 1. MODULOS

In [1]:
# Importa el módulo 'os' para interactuar con el sistema de archivos y rutas del sistema operativo
import os

# Importa 'pandas' como 'pd', una biblioteca potente para manipulación y análisis de datos mediante DataFrames
import pandas as pd

# Importa el módulo 'datetime' para trabajar con fechas y horas de manera eficiente
import datetime

# 2. RUTAS Y VARIABLES

In [2]:
# Se define la ruta de la carpeta donde se encuentran los archivos SAT
R_Sat = r"C:\Users\anthonycaicedo\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\BaseD\Sat_RC"

# Se define el nombre del archivo de salida SAT
SAT_SALIDA = "CONSOLIDADO_SAT_RC.TXT"

# Se construye la ruta completa del archivo de salida SAT utilizando una f-string
R_Salida = Fr"C:\Users\anthonycaicedo\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\BaseD\{SAT_SALIDA}"

# 3 UNIFICACION ARCHIVOS

In [3]:
import glob

# Explicación no técnica:
# 1. Vamos a buscar todos los archivos .txt dentro de la carpeta R_Sat y sus subcarpetas.
# 2. Cada archivo tiene datos separados por | y un encabezado (nombres de columnas).
# 3. El nombre del archivo contiene la fecha, la cual extraemos y la ponemos como la primera columna en cada registro.
# 4. Unimos todos los archivos en un solo archivo, eliminando los encabezados repetidos y renombrando las columnas como COL1, COL2, etc.
# 5. Guardamos el resultado en un nuevo archivo .txt, usando el formato ANSI y el separador |.


# Buscar todos los archivos .txt en la ruta y subcarpetas
archivos = glob.glob(os.path.join(R_Sat, '**', '*.txt'), recursive=True)

dataframes = []

for archivo in archivos:
    # Extraer la fecha del nombre del archivo
    nombre = os.path.basename(archivo)
    partes = nombre.split('-')
    if len(partes) >= 4:
        anio = partes[1]
        mes = partes[2]
        dia = partes[3].split('.')[0]
        fecha_archivo = f"{dia.zfill(2)}-{mes.zfill(2)}-{anio}"
    else:
        print(f"Alerta: El archivo '{nombre}' no tiene el formato esperado. Ejecución pausada.")
        break  # Detener la ejecución si se encuentra un archivo con formato inesperado

    # Leer el archivo como texto, sin importar el tipo de dato (todo como str)
    df = pd.read_csv(archivo, sep='|', dtype=str, encoding='ansi')
    df = df.iloc[1:]  # Eliminar el encabezado repetido si existe en los datos

    # Agregar la columna de fecha al inicio
    df.insert(0, 'fecha_archivo', fecha_archivo)
    dataframes.append(df)

# Unir todos los dataframes
df_total = pd.concat(dataframes, ignore_index=True)

# Renombrar las columnas a COL1, COL2, ...
df_total.columns = [f'COL{i+1}' for i in range(df_total.shape[1])]

# Guardar el resultado en el archivo de salida, usando | y formato ANSI
df_total.to_csv(R_Salida, sep='|', index=False, header=True, encoding='ansi')